In [151]:
import os

In [152]:
os.chdir("C:/Users/anand/Desktop/reume_projet/AutoPrice-AI")

In [153]:
import pandas as pd


In [154]:
data = pd.read_csv("artifacts/data_ingestion/dataset/cardatav_02.csv")
data.head()

,make,model,km_driven,ownership,transmission,engine,fuel,city,reg_year_int,car_age,price
0,Skoda,Laura 1.8 TSI Ambiente,60000.0,Fourth Owner,Manual,1798.0,Petrol,Jaipur,2009.0,16.0,200000.0
1,Skoda,Superb Elegance 2.0 TDI CR AT,120000.0,Second Owner,Automatic,1968.0,Diesel,Jaipur,2012.0,13.0,340000.0
2,Skoda,Kodiaq 2.0 TDI Style,73000.0,First Owner,Automatic,1968.0,Diesel,Jaipur,2020.0,5.0,2300000.0
3,Skoda,Kodiaq 2.0 TDI Style,108201.0,First Owner,Automatic,1968.0,Diesel,Jaipur,2018.0,7.0,1990000.0
4,Skoda,Kodiaq L & K BSVI,52000.0,First Owner,Automatic,1984.0,Petrol,Jaipur,2022.0,3.0,3100000.0


In [155]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2294 entries, 0 to 2293
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   make          2294 non-null   object 
 1   model         2294 non-null   object 
 2   km_driven     2277 non-null   float64
 3   ownership     2294 non-null   object 
 4   transmission  2294 non-null   object 
 5   engine        2294 non-null   float64
 6   fuel          2294 non-null   object 
 7   city          2294 non-null   object 
 8   reg_year_int  2282 non-null   float64
 9   car_age       2282 non-null   float64
 10  price         2292 non-null   float64
dtypes: float64(5), object(6)
memory usage: 197.3+ KB


In [156]:
data.isnull().sum()

make             0
model            0
km_driven       17
ownership        0
transmission     0
engine           0
fuel             0
city             0
reg_year_int    12
car_age         12
price            2
dtype: int64

In [157]:
data.shape


(2294, 11)

In [158]:
data.columns

Index(['make', 'model', 'km_driven', 'ownership', 'transmission', 'engine',
       'fuel', 'city', 'reg_year_int', 'car_age', 'price'],
      dtype='object')

In [159]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [160]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [161]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [162]:
import os
from mlProject import logger

In [163]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [164]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-06-14 04:11:53,857: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-14 04:11:53,860: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-14 04:11:53,862: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-14 04:11:53,864: INFO: common: created directory at: artifacts]
[2025-06-14 04:11:53,867: INFO: common: created directory at: artifacts/data_validation]
